In [1]:
import os
import pandas as pd
import json 
from openai import OpenAI

# CONFIGURACIÓN DE LLM Y DATOS 

def load_api_key_from_file(filepath: str) -> str:
    """Lee la clave API de OpenAI desde el archivo de texto en la ruta especificada."""
    try:
        with open(filepath, 'r') as f:
            # Lee la primera línea y elimina cualquier espacio en blanco o salto de línea
            api_key = f.readline().strip()
            if not api_key:
                raise ValueError("El archivo de clave API está vacío.")
            return api_key
    except FileNotFoundError:
        raise FileNotFoundError(f"Error: El archivo de clave API no se encontró en la ruta: {filepath}")
    except Exception as e:
        raise Exception(f"Error al leer el archivo de clave API: {e}")

# RUTA DONDE EL USUARIO GUARDA SU CLAVE API
API_KEY_FILEPATH = r"C:\Users\Acer\Desktop\GEN AI\Udemy\OPEN AI y API key\API key.txt"

try:
    # 1. Cargar la clave API
    api_key = load_api_key_from_file(API_KEY_FILEPATH)
    # 2. Inicializar el cliente con la clave cargada
    llm_client = OpenAI(api_key=api_key)
    LLM_READY = True
    print("Cliente de OpenAI inicializado correctamente.")
except Exception as e:
    # Si la clave o el archivo fallan, el script continuará en modo teórico
    print(f"Advertencia: Error al inicializar el cliente OpenAI. {e}")
    print("El código correrá en modo teórico. Asegúrate de que la ruta y la clave en el archivo sean correctas.")
    LLM_READY = False

MODEL_NAME = "gpt-4-turbo"

# RUTA ARCHIVO CESANTIAS_cAUSADAS
DATA_FILE = r"C:\Users\Acer\Desktop\Prueba técnica SUMMA\cesancias_causadas.xlsx" 

# Mapeo de la función 
TOOL_MAP = {}

Cliente de OpenAI inicializado correctamente.


In [2]:
# --- DEFINICIÓN DE HERRAMIENTAS (TOOL USE / FUNCTION CALLING - MCPs) ---
def get_severance_pay_info(document_id: int) -> str:
    """
    Busca información de cesantías (monto y mes causado) para un empleado
    específico usando su número de documento (ID), simulando la revisión del archivo de datos.
    """
    try:
        df = pd.read_excel(DATA_FILE) 
        
        df['Documento'] = pd.to_numeric(df['Documento'], errors='coerce').fillna(0).astype(int)
        record = df[df['Documento'] == document_id]

        if record.empty:
            return f"Error: No se encontró información de cesantías para el Documento {document_id}."

        amount = record['Censatias'].iloc[0]
        month = record['Mes'].iloc[0]
        
        # cambio de forato del monto
        # Se mantiene la corrección de sintaxis del print con una variable temporal
        amount_formatted = f"${amount:,.0f}".replace(",", "_TEMP_").replace(".", ",").replace("_TEMP_", ".")
        
        return f"Información encontrada para Documento {document_id}:\nCesantías Causadas: {amount_formatted}\nMes de Causa: {month}"

    except FileNotFoundError:
        # Se incluye la variable DATA_FILE, que ahora contiene la ruta completa, para debug
        return f"Error: Archivo de datos '{DATA_FILE}' no encontrado. Verifique la ruta "
    except Exception as e:
        return f"Error al procesar la búsqueda en el archivo: {e}"

# Esquema de la función para que el LLM entienda cómo y cuándo usarla
TOOL_SCHEMA = [{
    "type": "function",
    "function": {
        "name": "get_severance_pay_info",
        "description": "Busca la información de cesantías causadas (monto y mes) para un empleado específico, utilizando su número de documento (ID). Útil para responder preguntas sobre datos privados.",
        "parameters": {
            "type": "object",
            "properties": {
                "document_id": {
                    "type": "integer",
                    "description": "El número de documento de identificación del empleado (por ejemplo, 131422)."
                }
            },
            "required": ["document_id"]
        }
    }
}]

# Actualizar el mapeo de la función (necesario para el flujo de la celda 2)
TOOL_MAP["get_severance_pay_info"] = get_severance_pay_info


# Logica del agente(ORQUESTADOR DE MCPs) 

def run_hr_agent(user_question: str) -> str:
    """
    Función principal del agente. Decide si responde con conocimiento base (Opción a) o
    usa la herramienta para acceder a datos (Opción b), implementando la lógica de MCPs.
    """
    if not LLM_READY:
        # Modo Teórico si la clave API falla
        if "documento" in user_question.lower() or "id" in user_question.lower():
            return f"--- PROCESO TEÓRICO: El Agente decidiría que esta pregunta se responde con la Opción B (Acceso a Datos) y llamaría a la función 'get_severance_pay_info'."
        else:
            return f"--- PROCESO TEÓRICO: El Agente decidiría que esta pregunta se responde con la Opción A (Conocimiento Base) sin usar herramientas."

    # 1. Definir la System Instruction y la Mensajería Inicial
    system_instruction = (
        "Eres un agente de soporte de Gestión Humana de una compañía en Colombia. "
        "Tus respuestas deben ser concisas, profesionales y amigables. "
        "Tu objetivo es asistir al usuario con información de cesantías. "
        "Si la pregunta requiere información de cesantías de un empleado específico usando su número de documento (ID), "
        "utiliza **obligatoriamente** la herramienta 'get_severance_pay_info' (Opción b) con el ID extraído de la pregunta."
    )
    
    messages = [
        {"role": "system", "content": system_instruction},
        {"role": "user", "content": user_question}
    ]
    
    # 2. Primera Llamada al LLM: ¿Respuesta o Llamada a Función?
    response = llm_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
        tools=TOOL_SCHEMA,
        tool_choice="auto" 
    )

    response_message = response.choices[0].message
    
    # 3. Ciclo de Ejecución de la Herramienta (MCPs)
    if response_message.tool_calls:
        # El modelo solicitó usar la herramienta (Acceso a Datos)
        
        # Guardar la solicitud de la función en el historial
        messages.append(response_message)
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            
            if function_name in TOOL_MAP:
                function_args = json.loads(tool_call.function.arguments)
                doc_id = function_args.get('document_id')

                print(f"[PROCESO DE PENSAMIENTO]: El agente decidió usar conocimiento específico para el Documento ID: {doc_id}.")

                # Ejecutar la función Python real (Acción de la Herramienta)
                tool_output = TOOL_MAP[function_name](document_id=doc_id)
                
                # Solución para el SyntaxError
                tool_output_oneline = tool_output.replace('\n', ' | ')
                print(f"[RESULTADO DE FUNCIÓN]: {tool_output_oneline}")

                # Enviar el resultado de la función de vuelta al LLM
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": tool_output,
                    }
                )
        
        # 4. Segunda Llamada al LLM: Generar la respuesta final con el resultado de la herramienta
        response = llm_client.chat.completions.create(
            model=MODEL_NAME,
            messages=messages,
        )
        return response.choices[0].message.content
    
    # Si no hay llamada a función, el modelo responde directamente (Opción a - Conocimiento Base)
    print("[PROCESO DE PENSAMIENTO]: El agente decidió responder con Conocimiento Base.")
    return response_message.content

In [4]:
# --- PRUEBA DE CONCEPTO CON 4 PREGUNTAS ---

# 4 Preguntas que prueban ambas opciones (a y b)
test_questions = [
    "Qué son las cesantías?", 
    "Cómo puedo hacer el retiro de mis cesantías?", 
    "Dime los documentos de las personas que tengan mas de 5 millones en esantías", 
    "Cuantas cesantias tengo? mi documento es 141601",
    "cuantas cesantías tengo mi documento es 123456"
]

print("-" * 100)
print(f"INICIO DE PRUEBA DE CONCEPTO - PROYECTO AGENTE ({MODEL_NAME})")
print("-" * 100)

for i, q in enumerate(test_questions):
    print(f"\n--- PREGUNTA {i+1} ---")
    print(f"USUARIO: {q}")
    
    try:
        final_response = run_hr_agent(q)
        print(f"AGENTE: {final_response}")
        print("-" * 30)

    except Exception as e:
        # Captura cualquier error de ejecución
        print(f"\n[ERROR DE EJECUCIÓN INESPERADO]: No se pudo completar la prueba. Detalle: {e}")
        break

----------------------------------------------------------------------------------------------------
INICIO DE PRUEBA DE CONCEPTO - PROYECTO AGENTE (gpt-4-turbo)
----------------------------------------------------------------------------------------------------

--- PREGUNTA 1 ---
USUARIO: Qué son las cesantías?
[PROCESO DE PENSAMIENTO]: El agente decidió responder con Conocimiento Base.
AGENTE: Las cesantías son una prestación social en Colombia que tiene como objetivo principal proteger al trabajador en caso de desempleo. Funcionan como una forma de ahorro forzoso, ya que se trata de un dinero que el empleador consigna anualmente en un fondo de cesantías a nombre del empleado. Esta suma equivale al 12% del salario anual del trabajador.

Las cesantías se pueden retirar bajo ciertas condiciones, como en el caso de terminación del contrato de trabajo, o pueden utilizarse para financiar proyectos como vivienda, educación o en situaciones especiales. La liquidación de estas se realiza un